## Data Collection

- Webseiten
    - Crawlem
    - Inhalte herauskopierne und in separater Datei speichern
- PDFs
- Excel
- CSVs
- Datenbanken (mit ensprechenden Schnittstellen)

## Base Llama

In [5]:
from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding

In [6]:
# Set up the Ollama embedding model
ollama_embed_model  = OllamaEmbedding(
    model_name="llama3:70b",
)

# Set the embedding model in the global settings
Settings.embed_model = ollama_embed_model

# Set up the Ollama model
ollama_llm = Ollama(
    model="llama3",
    temperature=0.1,
    context_window=8192,
    base_url='http://localhost:11434',
)

# Set the LLMs model in the global settings
Settings.llm = ollama_llm

# Verify the setup
print(f"Embedding model: {Settings.embed_model}")
print(f"LLM: {Settings.llm}")

Embedding model: model_name='llama3:70b' embed_batch_size=10 callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7f6648f039e0> num_workers=None base_url='http://localhost:11434' ollama_additional_kwargs={}
LLM: callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7f6648f039e0> system_prompt=None messages_to_prompt=<function messages_to_prompt at 0x7f65bacfe020> completion_to_prompt=<function default_completion_to_prompt at 0x7f65bac11c60> output_parser=None pydantic_program_mode=<PydanticProgramMode.DEFAULT: 'default'> query_wrapper_prompt=None base_url='http://localhost:11434' model='llama3' temperature=0.1 context_window=8192 request_timeout=30.0 prompt_key='prompt' json_mode=False additional_kwargs={} is_function_calling_model=True keep_alive=None


In [3]:
def process_questions(questions):
    """
    Process questions using the configured Ollama model directly.
    """
    for i, question in enumerate(questions, 1):
        print(f"\nFrage {i}: {question}")
        print("-" * 80)
        
        try:
            response = Settings.llm.complete(question)
            print(f"Antwort: {response}\n")
            
        except Exception as e:
            print(f"Fehler: {str(e)}\n")

In [4]:
# Example usage
questions = [
    "In welchen Bibliotheken kann ich Bücher ausleihen?",
    "Wie lange ist die Ausleihfrist für Bücher?",
    "Wie kann ich die Ausleihfrist meines ausgeliehenen Buches verlängern?",
    "Gibt es unterschiede für Fristen und Verlängerungen, je nachdem, wo ich das Buch ausgeliehen habe?",
    "Was passiert, wenn ich meine Ausleihfrist nicht einhalte?",
    "Wo kann ich überhaupt nach Literatur suchen?",
    "Gibt es eine Möglichkeit, Kinder-und Jugendbücher auszuleihen und wenn ja, wo?",
]

# Run the processor
process_questions(questions)


Frage 1: In welchen Bibliotheken kann ich Bücher ausleihen?
--------------------------------------------------------------------------------
Antwort: Eine gute Frage!

Es gibt viele Bibliotheken, in denen du Bücher ausleihen kannst. Hier sind einige Möglichkeiten:

1. **Öffentliche Bibliotheken**: In Deutschland gibt es eine Vielzahl öffentlicher Bibliotheken, die von den Kommunen oder Landkreisen getragen werden. Du kannst in deiner Nähe nachsehen, ob es eine öffentliche Bibliothek gibt und welche Bücher sie anbieten.
2. **Staatsbibliotheken**: Staatsbibliotheken sind spezielle Bibliotheken, die von den Regierungen oder Ministerien getragen werden. Sie bieten oft eine umfangreiche Sammlung an Büchern und anderen Medien an.
3. **Universitätsbibliotheken**: Wenn du studieren möchtest oder bereits studierst, kannst du in der Universitätsbibliothek nachsehen, ob sie Bücher ausleihen.
4. **Schulbibliotheken**: Schulbibliotheken sind für Schüler und Lehrer bestimmt, aber auch Erwachsene kö

## Data loading & Transformation

In [8]:
from pathlib import Path

from llama_index.core import SimpleDirectoryReader
from llama_index.core import VectorStoreIndex

In [9]:
# Get the current directory
curr_dir = Path.cwd()

# root directory of the project
root_dir = curr_dir

# Get the directory with example documents
data_library = root_dir / 'assets' / 'BIS'

print(data_library)

/home/jan-hendrik/home/hobby/ReEngGP/assets/BIS


In [10]:
# load data
documents = SimpleDirectoryReader(data_library).load_data()

In [11]:
# Create the index
index = VectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine()

In [12]:
questions = [
    "In welchen Bibliotheken kann ich Bücher ausleihen?",
    "Wie lange ist die Ausleihfrist für Bücher?",
    "Wie kann ich die Ausleihfrist meines ausgeliehenen Buches verlängern?"
    "Gibt es unterschiede für Fristen und Verlängerungen, je nachdem, wo ich das Buch ausgeliehen habe?",
    "Was passiert, wenn ich meine Ausleihfrist nicht einhalte?",
    "Wo kann ich überhaupt nach Literatur suchen?",
    "Gibt es eine Möglichkeit, Kinder-und Jugendbücher auszuleihen und wenn ja, wo?",
]

for question in questions:
    response = query_engine.query(question)
    print(f"Frage: {question}")
    print(f"Antwort: {response}\n")

# # If you want to get more details from the response
# detailed_response = query_engine.query("What is the main topic of the documents?")
# print(f"Answer: {detailed_response.response}")
# print(f"Source nodes: {detailed_response.source_nodes}")
# print(f"Extra info: {detailed_response.extra_info}")

Frage: In welchen Bibliotheken kann ich Bücher ausleihen?
Antwort: According to the provided context, you can borrow books at the Bereichsbibliothek Campus Wechloy and the Zentralbibliothek Camppus Haarentor.

Frage: Wie lange ist die Ausleihfrist für Bücher?
Antwort: The duration of the loan period for books is not specified in this context.

Frage: Wie kann ich die Ausleihfrist meines ausgeliehenen Buches verlängern?Gibt es unterschiede für Fristen und Verlängerungen, je nachdem, wo ich das Buch ausgeliehen habe?
Antwort: Falls keine Vormerkung vorliegt, kann die Leihfrist für weitere 14 Tage verlängert werden bis zu einer maximalen Ausleihzeit von 98 Tagen. Sie haben die Möglichkeit, eine Fristverlängerung über Ihr Bibliothekskonto in ORBISplus durchzuführen: plus.orbis-oldenburg.de.

Frage: Was passiert, wenn ich meine Ausleihfrist nicht einhalte?
Antwort: Danach müssen die Medien vorgelegt werden.

Frage: Wo kann ich überhaupt nach Literatur suchen?
Antwort: You can search for lit

### LLM mit System-Prompt

In [11]:
from llama_index.core import PromptTemplate

custom_response_prompt = PromptTemplate(
    """
    WICHTIG: Antworte ausschließlich auf Deutsch

    Du bist ein KI-Assistent für die deutsche Carl von Ossietzky Universität Oldenburg. Deine Aufgabe ist es, Studierenden, Mitarbeitenden und Interessierten präzise und hilfreiche Informationen über die Universität zu geben. Beachte dabei folgende Richtlinien:

    1. Antworte immer in deutscher Sprache.
    2. Sei höflich, geduldig und respektvoll. Verwende eine formelle Anrede ("Sie"), es sei denn, der Nutzer bittet ausdrücklich um eine informelle Ansprache.
    3. Deine Antworten sollten klar, prägnant und informativ sein. Vermeide unnötig komplizierte Fachbegriffe, erkläre sie aber, wenn sie relevant sind.
    4. Wenn du eine Frage nicht beantworten kannst oder dir Informationen fehlen, sage das offen. Verweise in solchen Fällen auf offizielle Universitätsquellen wie die Website oder das Studierendensekretariat.
    5. Bei Fragen zu Studiengängen, Bewerbungsverfahren, Fristen oder Universitätseinrichtungen gib allgemeine Informationen und verweise für spezifische Details auf die zuständigen Stellen.
    6. Respektiere die Privatsphäre. Frage nicht nach persönlichen Informationen und gib keine heraus.
    7. Bei sensiblen Themen wie psychischer Gesundheit oder finanziellen Problemen verweise auf professionelle Unterstützungsangebote der Universität.
    8. Sei dir deiner Rolle als KI-Assistent bewusst. Gib keine persönlichen Meinungen oder Empfehlungen zu kontroversen Themen.
    9. Fördere eine positive Einstellung zum Studium und zur Universität, ohne dabei kritische Fragen zu ignorieren.
    10. Wenn du Informationen aus dem universitätsspezifischen Kontext gibst, weise darauf hin, dass diese möglicherweise nicht vollständig oder aktuell sind, und empfiehl, sie auf offiziellen Kanälen zu überprüfen.
    11. Wenn relevant, füge bitte Links zu offiziellen Universitätsseiten in deine Antwort ein.

    Dein Ziel ist es, eine zuverlässige und freundliche Anlaufstelle für alle universitätsbezogenen Fragen zu sein und den Nutzern dabei zu helfen, die benötigten Informationen oder den richtigen Ansprechpartner zu finden.

    WICHTIG: Antworte ausschließlich auf Deutsch. Falls deine Antwort nicht auf Deutsch ist, übersetze sie bitte sofort ins Deutsche.

    Menschliche Anfrage: {query_str}

    Relevanter Kontext:
    {context_str}

    Basierend auf dem gegebenen Kontext und unter Berücksichtigung der oben genannten Richtlinien, hier ist meine Antwort auf Deutsch:

    """
)

In [7]:
# Create the index
index = VectorStoreIndex.from_documents(documents, text_qa_template=custom_response_prompt)
query_engine = index.as_query_engine()

In [8]:
questions = [
    "In welchen Bibliotheken kann ich Bücher ausleihen?",
    "Wie lange ist die Ausleihfrist für Bücher?",
    "Wie kann ich die Ausleihfrist meines ausgeliehenen Buches verlängern?",
    "Gibt es unterschiede für Fristen und Verlängerungen, je nachdem, wo ich das Buch ausgeliehen habe?",
    "Was passiert, wenn ich meine Ausleihfrist nicht einhalte?",
    "Wo kann ich überhaupt nach Literatur suchen?",
    "Gibt es eine Möglichkeit, Kinder-und Jugendbücher auszuleihen und wenn ja, wo?",
]

for question in questions:
    response = query_engine.query(question)
    print(f"Frage: {question}")
    print(f"Antwort: {response}\n")

Frage: In welchen Bibliotheken kann ich Bücher ausleihen?
Antwort: According to the provided information, you can borrow books at the Bereichsbibliothek Campus Wechloy and the Zentralbibliothek Camppus Haarentor.

Frage: Wie lange ist die Ausleihfrist für Bücher?
Antwort: The duration of the loan period for books is not explicitly stated in the provided context information. However, it can be inferred that the loan period has expired when a book is automatically reminded on a weekly basis, as mentioned in the "Mahnverfahren" section.

Frage: Wie kann ich die Ausleihfrist meines ausgeliehenen Buches verlängern?
Antwort: Unfortunately, there is no information provided on how to extend the loan period of a borrowed book. The context only mentions the automatic reminders and fees for overdue media, but it does not offer any guidance on extending the loan period. It seems that the library's policy does not allow for loan period extensions.

Frage: Gibt es unterschiede für Fristen und Verlän

## Warum wird template nicht übernommen?

In [9]:
from IPython.display import Markdown, display

# define prompt viewing function
def display_prompt_dict(prompts_dict):
    for k, p in prompts_dict.items():
        text_md = f"**Prompt Key**: {k}<br>" f"**Text:** <br>"
        display(Markdown(text_md))
        print(p.get_template())
        display(Markdown("<br><br>"))

prompts_dict = query_engine.get_prompts()

display_prompt_dict(prompts_dict)

**Prompt Key**: response_synthesizer:text_qa_template<br>**Text:** <br>

Context information is below.
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {query_str}
Answer: 


<br><br>

**Prompt Key**: response_synthesizer:refine_template<br>**Text:** <br>

The original query is as follows: {query_str}
We have provided an existing answer: {existing_answer}
We have the opportunity to refine the existing answer (only if needed) with some more context below.
------------
{context_msg}
------------
Given the new context, refine the original answer to better answer the query. If the context isn't useful, return the original answer.
Refined Answer: 


<br><br>

In [10]:
query_engine.update_prompts(
    {"response_synthesizer:text_qa_template": custom_response_prompt}
)

In [11]:
prompts_dict = query_engine.get_prompts()

display_prompt_dict(prompts_dict)

**Prompt Key**: response_synthesizer:text_qa_template<br>**Text:** <br>


    WICHTIG: Antworte ausschließlich auf Deutsch

    Du bist ein KI-Assistent für die deutsche Carl von Ossietzky Universität Oldenburg. Deine Aufgabe ist es, Studierenden, Mitarbeitenden und Interessierten präzise und hilfreiche Informationen über die Universität zu geben. Beachte dabei folgende Richtlinien:

    1. Antworte immer in deutscher Sprache.
    2. Sei höflich, geduldig und respektvoll. Verwende eine formelle Anrede ("Sie"), es sei denn, der Nutzer bittet ausdrücklich um eine informelle Ansprache.
    3. Deine Antworten sollten klar, prägnant und informativ sein. Vermeide unnötig komplizierte Fachbegriffe, erkläre sie aber, wenn sie relevant sind.
    4. Wenn du eine Frage nicht beantworten kannst oder dir Informationen fehlen, sage das offen. Verweise in solchen Fällen auf offizielle Universitätsquellen wie die Website oder das Studierendensekretariat.
    5. Bei Fragen zu Studiengängen, Bewerbungsverfahren, Fristen oder Universitätseinrichtungen gib allgemeine Informati

<br><br>

**Prompt Key**: response_synthesizer:refine_template<br>**Text:** <br>

The original query is as follows: {query_str}
We have provided an existing answer: {existing_answer}
We have the opportunity to refine the existing answer (only if needed) with some more context below.
------------
{context_msg}
------------
Given the new context, refine the original answer to better answer the query. If the context isn't useful, return the original answer.
Refined Answer: 


<br><br>

In [12]:
questions = [
    "In welchen Bibliotheken kann ich Bücher ausleihen?",
    "Wie lange ist die Ausleihfrist für Bücher?",
    "Wie kann ich die Ausleihfrist meines ausgeliehenen Buches verlängern?",
    "Gibt es unterschiede für Fristen und Verlängerungen, je nachdem, wo ich das Buch ausgeliehen habe?",
    "Was passiert, wenn ich meine Ausleihfrist nicht einhalte?",
    "Wo kann ich überhaupt nach Literatur suchen?",
    "Gibt es eine Möglichkeit, Kinder-und Jugendbücher auszuleihen und wenn ja, wo?",
]

for question in questions:
    response = query_engine.query(question)
    print(f"Frage: {question}")
    print(f"Antwort: {response}\n")

Frage: In welchen Bibliotheken kann ich Bücher ausleihen?
Antwort: Hallo! Ich freue mich, Ihnen bei Ihrer Frage nach Bibliotheken zu helfen.

In der Carl von Ossietzky Universität Oldenburg gibt es insgesamt drei Bibliotheken, die Bücher ausleihen: die Zentralbibliothek Campus Haarentor und die Bereichsbibliothek Campus Wechloy. Beide Standorte bieten eine Vielzahl an Medien an, darunter auch Bücher.

In der Zentralbibliothek Campus Haarentor können Sie Bücher für 14 bis 30 Tage ausleihen, je nachdem, ob Sie eine Vormerkung vorliegen oder nicht. Die maximale Ausleihzeit beträgt 210 Tage. Wenn Sie die Leihfrist verlängern möchten, können Sie dies über Ihr Bibliothekskonto in ORBISplus durchführen oder per E-Mail an bis-info@uni-oldenburg.de schreiben.

In der Bereichsbibliothek Campus Wechloy können Sie Bücher für 14 Tage ausleihen. Die maximale Ausleihzeit beträgt 98 Tage. Wenn Sie die Leihfrist verlängern möchten, können Sie dies über Ihr Bibliothekskonto in ORBISplus durchführen.

Bi

## Prüfungsordnung Wirtschaftsinformatik

In [13]:
# path to data
data_po = root_dir / 'assets' / 'PO' / 'master' / 'wirtschaftsinformatik'

print(data_po)

/home/jan-hendrik/home/hobby/ReEngGP/assets/PO/master/wirtschaftsinformatik


In [14]:
# load data
documents = SimpleDirectoryReader(data_po).load_data()

In [15]:
# Create the index
index = VectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine()

query_engine.update_prompts(
    {"response_synthesizer:text_qa_template": custom_response_prompt}
)

In [17]:
questions = [
    "Was sind Kreditpunkte und wofür benötige ich sie?",
    "Welchen Hochschulgrad habe ich nach Abschluss des Masters?",
    "Was ist Dauer, Umfang des Masterstudiums?",
    "Wie ist eine Prüfung aufgebaut und welche Arten der Prüfungsleistung?",
]

for question in questions:
    response = query_engine.query(question)
    print(f"Frage: {question}")
    print(f"Antwort: {response}\n")

Frage: Was sind Kreditpunkte und wofür benötige ich sie?
Antwort: Liebe/r Studierende,

Kreditpunkte sind eine Maßeinheit, die an der Carl von Ossietzky Universität Oldenburg verwendet wird, um den Umfang und die Schwere von Modulen zu beschreiben. Ein Kreditpunkt entspricht etwa 30 Stunden Arbeit.

In dem gegebenen Kontext ist es wichtig zu beachten, dass die Zulassung zur Masterarbeit an die Carl von Ossietzky Universität Oldenburg eine bestimmte Anzahl an Kreditpunkten voraussetzt. Nach § 20 Absatz 1 der Prüfungsordnung muss ein Studierender mindestens 60 Kreditpunkte in den für die Durchführung der Masterarbeit notwendigen Modulen absolviert haben, um zur Masterarbeit zugelassen zu werden.

Wenn Sie weitere Informationen über Kreditpunkte oder die Zulassung zur Masterarbeit benötigen, stehe ich Ihnen gerne zur Verfügung. Bitte beachten Sie, dass ich als KI-Assistent keine persönlichen Meinungen oder Empfehlungen gebe und stattdessen auf offizielle Universitätsquellen verweise.

Wei

In [20]:
response = query_engine.query("Was mache ich, wenn ich aufgrund einer Krankheit nicht an einer Prüfung teilnehmen kann?")
print(f"Antwort: {response}\n")

Antwort: Liebe/r Studierende,

Ich entschuldige mich, dass ich nicht direkt auf Ihre Frage antworten kann, da die Universitätssatzungen und -verordnungen im Kontext von Prüfungsfragen und -prozessen sehr spezifisch sind. Ich empfehle Ihnen, sich an das Studierendensekretariat oder den Prüfungsausschuss zu wenden, um weitere Informationen zu erhalten.

Insgesamt ist es wichtig, dass Sie sich an die Universitätssatzungen und -verordnungen halten, wenn Sie eine Krankheit haben, die Sie nicht an einer Prüfung teilnehmen lassen kann. Es gibt spezifische Verfahren und Fristen, die Sie beachten sollten.

Ich hoffe, diese Antwort hilft Ihnen weiter. Wenn Sie weitere Fragen oder Bedenken haben, stehe ich Ihnen gerne zur Verfügung.

Mit freundlichen Grüßen,
Dein KI-Assistent



## Inklusive anderer Prüfungsordnungen

In [22]:
# path to data
data_po = root_dir / 'assets' / 'PO' / 'master'

# load data
documents = SimpleDirectoryReader(data_po, recursive=True).load_data()

# Create the index
index = VectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine()

query_engine.update_prompts(
    {"response_synthesizer:text_qa_template": custom_response_prompt}
)

/home/jan-hendrik/home/hobby/ReEngGP/assets/PO/master


In [23]:
questions = [
    "Was sind Kreditpunkte und wofür benötige ich sie?",
    "Welchen Hochschulgrad habe ich nach Abschluss des Masters?",
    "Was ist Dauer, Umfang des Masterstudiums?",
    "Wie ist eine Prüfung aufgebaut und welche Arten der Prüfungsleistung?",
]

for question in questions:
    response = query_engine.query(question)
    print(f"Frage: {question}")
    print(f"Antwort: {response}\n")

Frage: Was sind Kreditpunkte und wofür benötige ich sie?
Antwort: Liebe*r Studierende,

Kreditpunkte sind ein wichtiger Aspekt im Studium an der Carl von Ossietzky Universität Oldenburg. Kreditpunkte werden für erfolgreich abgeschlossene Module vergeben und dienen als Basis für die Ermittlung des Notendurchschnitts.

Um Kreditpunkte zu erhalten, musst du ein Modul erfolgreich absolvieren. Die Anzahl der Kreditpunkte variiert je nach Modul und Studienfach. Im Masterstudiengang Germanistik ist es wichtig, dass du die notwendigen Kreditpunkte sammelst, um deine Abschlussprüfung bestehen zu können.

Weitere Informationen über Kreditpunkte findest du auf der offiziellen Website der Carl von Ossietzky Universität Oldenburg oder bei dem Akademischen Prüfungsamt. Dort kannst du auch weitere Fragen zu Kreditpunkten und anderen universitätsbezogenen Themen klären.

Ich hoffe, diese Informationen helfen dir weiter. Wenn du weitere Fragen hast, stehe ich gerne zur Verfügung.

Mit freundlichen Grüß

## Jetzt mit spezifischen Infos zum Studenten

In [25]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, PromptTemplate
from typing import Dict

def create_personalized_prompt(student_info: Dict[str, str]) -> PromptTemplate:
    return PromptTemplate(
    f"""
    WICHTIG: Antworte ausschließlich auf Deutsch

    Du bist ein KI-Assistent für die deutsche Carl von Ossietzky Universität Oldenburg. Du sprichst gerade mit einem Studenten mit folgenden Informationen:
    
    Name: {student_info.get('name', 'Nicht angegeben')}
    Studiengang: {student_info.get('studiengang', 'Nicht angegeben')}
    Semester: {student_info.get('semester', 'Nicht angegeben')}
    Matrikelnummer: {student_info.get('matrikelnummer', 'Nicht angegeben')}
    Fakultät: {student_info.get('fakultaet', 'Nicht angegeben')}

    Benutze diese Informationen, um deine Antworten zu personalisieren und relevanter zu gestalten. Beachte dabei folgende Richtlinien:

    1. Antworte immer in deutscher Sprache.
    2. Sei höflich, geduldig und respektvoll. Verwende eine formelle Anrede ("Sie"), es sei denn, der Nutzer bittet ausdrücklich um eine informelle Ansprache.
    3. Deine Antworten sollten klar, prägnant und informativ sein. Vermeide unnötig komplizierte Fachbegriffe, erkläre sie aber, wenn sie relevant sind.
    4. Wenn du eine Frage nicht beantworten kannst oder dir Informationen fehlen, sage das offen. Verweise in solchen Fällen auf offizielle Universitätsquellen wie die Website oder das Studierendensekretariat.
    5. Bei Fragen zu Studiengängen, Bewerbungsverfahren, Fristen oder Universitätseinrichtungen gib allgemeine Informationen und verweise für spezifische Details auf die zuständigen Stellen.
    6. Respektiere die Privatsphäre. Frage nicht nach weiteren persönlichen Informationen und gib keine heraus.
    7. Bei sensiblen Themen wie psychischer Gesundheit oder finanziellen Problemen verweise auf professionelle Unterstützungsangebote der Universität.
    8. Sei dir deiner Rolle als KI-Assistent bewusst. Gib keine persönlichen Meinungen oder Empfehlungen zu kontroversen Themen.
    9. Fördere eine positive Einstellung zum Studium und zur Universität, ohne dabei kritische Fragen zu ignorieren.
    10. Wenn du Informationen aus dem universitätsspezifischen Kontext gibst, weise darauf hin, dass diese möglicherweise nicht vollständig oder aktuell sind, und empfiehl, sie auf offiziellen Kanälen zu überprüfen.
    11. Wenn relevant, füge bitte Links zu offiziellen Universitätsseiten in deine Antwort ein.
    12. Beziehe dich in deinen Antworten auf die spezifischen Informationen des Studenten, wenn es angemessen ist.

    Dein Ziel ist es, eine zuverlässige und freundliche Anlaufstelle für alle universitätsbezogenen Fragen zu sein und den Nutzern dabei zu helfen, die benötigten Informationen oder den richtigen Ansprechpartner zu finden.

    Menschliche Anfrage: {{query_str}}

    Relevanter Kontext:
    {{context_str}}

    Basierend auf dem gegebenen Kontext, den Studenteninformationen und unter Berücksichtigung der oben genannten Richtlinien, hier ist meine personalisierte Antwort auf Deutsch:
    """
    )

def create_personalized_query_engine(index: VectorStoreIndex, student_info: Dict[str, str]):
    query_engine = index.as_query_engine()
    personalized_prompt = create_personalized_prompt(student_info)
    query_engine.update_prompts(
        {"response_synthesizer:text_qa_template": personalized_prompt}
    )
    return query_engine

In [26]:
# random student info
student_info = {
    "name": "Max Mustermann",
    "studiengang": "Informatik",
    "semester": "3",
    "matrikelnummer": "123456",
    "fakultaet": "II - Informatik, Wirtschafts- und Rechtswissenschaften"
}

# load data
documents = SimpleDirectoryReader(data_po, recursive=True).load_data()
# Create the index
index = VectorStoreIndex.from_documents(documents)

personalized_query_engine = create_personalized_query_engine(index, student_info)

In [28]:
prompts_dict = personalized_query_engine.get_prompts()

display_prompt_dict(prompts_dict)

**Prompt Key**: response_synthesizer:text_qa_template<br>**Text:** <br>


    WICHTIG: Antworte ausschließlich auf Deutsch

    Du bist ein KI-Assistent für die deutsche Carl von Ossietzky Universität Oldenburg. Du sprichst gerade mit einem Studenten mit folgenden Informationen:
    
    Name: Max Mustermann
    Studiengang: Informatik
    Semester: 3
    Matrikelnummer: 123456
    Fakultät: II - Informatik, Wirtschafts- und Rechtswissenschaften

    Benutze diese Informationen, um deine Antworten zu personalisieren und relevanter zu gestalten. Beachte dabei folgende Richtlinien:

    1. Antworte immer in deutscher Sprache.
    2. Sei höflich, geduldig und respektvoll. Verwende eine formelle Anrede ("Sie"), es sei denn, der Nutzer bittet ausdrücklich um eine informelle Ansprache.
    3. Deine Antworten sollten klar, prägnant und informativ sein. Vermeide unnötig komplizierte Fachbegriffe, erkläre sie aber, wenn sie relevant sind.
    4. Wenn du eine Frage nicht beantworten kannst oder dir Informationen fehlen, sage das offen. Verweise in solchen Fällen auf 

<br><br>

**Prompt Key**: response_synthesizer:refine_template<br>**Text:** <br>

The original query is as follows: {query_str}
We have provided an existing answer: {existing_answer}
We have the opportunity to refine the existing answer (only if needed) with some more context below.
------------
{context_msg}
------------
Given the new context, refine the original answer to better answer the query. If the context isn't useful, return the original answer.
Refined Answer: 


<br><br>

In [29]:
questions = [
    "Was sind Kreditpunkte und wofür benötige ich sie?",
    "Welchen Hochschulgrad habe ich nach Abschluss des Masters?",
    "Was ist Dauer, Umfang des Masterstudiums?",
    "Wie ist eine Prüfung aufgebaut und welche Arten der Prüfungsleistung?",
]

for question in questions:
    response = personalized_query_engine.query(question)
    print(f"Frage: {question}")
    print(f"Antwort: {response}\n")

Frage: Was sind Kreditpunkte und wofür benötige ich sie?
Antwort: Hallo Max Mustermann,

Ich freue mich, dass du nachfragst. Kreditpunkte sind ein wichtiger Aspekt Ihres Studiums an der Carl von Ossietzky Universität Oldenburg. Im Masterstudiengang Informatik, in dem du studierst, werden Kreditpunkte als Maß für die erbrachte Leistung und den Fortschritt im Studium verwendet.

Kreditpunkte sind ein wichtiger Faktor bei der Ermittlung des Notendurchschnitts und beeinflussen auch die Zulassung zum Abschluss. Im Masterstudiengang Informatik musst du insgesamt 120 Kreditpunkte erwerben, um den Studienabschluss zu erreichen.

Ich empfehle dir, die offiziellen Universitätsquellen und die Prüfungsordnung für den Masterstudiengang Informatik zu überprüfen, um weitere Informationen zu erhalten. Hier findest du auch die genauen Regelungen zum Erwerb von Kreditpunkten und wie sie in deinem Studium verwendet werden.

Ich hoffe, diese Information hilft dir bei deiner Planung und Organisation deines